In [ ]:
#https://www.google.com/search?q=bonsai+tree&tbm=isch&ved=2ahUKEwjPgprniIjtAhXKneAKHZuABioQ2-cCegQIABAA&oq=bonsai+tree&gs_lcp=CgNpbWcQAzIHCAAQsQMQQzICCAAyAggAMgIIADICCAAyAggAMgQIABBDMgIIADICCAAyAggAUNfnlANY1-eUA2CN6ZQDaABwAHgAgAFIiAFIkgEBMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=OveyX8_9Fsq7ggebgZrQAg&bih=1329&biw=2560&rlz=1C1CHBF_enUS866US866&safe=strict

#https://towardsdatascience.com/image-scraping-with-python-a96feda8af2d

#https://www.bonsaiempire.com/tree-species

In [1]:
from selenium import webdriver
import time
import requests
import io
import os
from PIL import Image
import hashlib
from selenium.webdriver.common.keys import Keys

# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
DRIVER_PATH = "C:/Users/eberz/Documents/ChromeDriver/chromedriver"
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [2]:
wd.get('https://www.google.com/imghp?hl=EN')

In [3]:
search_box = wd.find_element_by_css_selector('input.gLFyf')
search_box.send_keys('bonsai tree')
search_box.send_keys(Keys.RETURN)


In [4]:
#new_url = "https://www.google.com/search?q=bonsai+tree&tbm=isch&ved=2ahUKEwjPgprniIjtAhXKneAKHZuABioQ2-cCegQIABAA&oq=bonsai+tree&gs_lcp=CgNpbWcQAzIHCAAQsQMQQzICCAAyAggAMgIIADICCAAyAggAMgQIABBDMgIIADICCAAyAggAUNfnlANY1-eUA2CN6ZQDaABwAHgAgAFIiAFIkgEBMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=OveyX8_9Fsq7ggebgZrQAg&bih=1329&biw=2560&rlz=1C1CHBF_enUS866US866&safe=strict"
new_url = "https://www.google.com/search?q=dwarf+jade+bonsai&tbm=isch&ved=2ahUKEwi24Oiym4rtAhVKBd8KHTLXCfMQ2-cCegQIABAA&oq=dwarf+jade+bonsai&gs_lcp=CgNpbWcQAzIECCMQJzICCAAyAggAMgIIADICCAAyAggAMgYIABAIEB4yBggAEAgQHjIGCAAQCBAeMgQIABAYOgUIABCxAzoECAAQQ1CSJVi-QWDfQ2gBcAB4AIABUYgB7wiSAQIxOJgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=KBe0X7asEMqK_AayrqeYDw&bih=1329&biw=2560&rlz=1C1CHBF_enUS866US866&safe=strict"

In [5]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = new_url

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            #return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [6]:
# Never need to change. Unless preference for png over jpeg potentially?
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [7]:
# Function that puts the above two all together
def search_and_download(search_term:str,driver_path:str,target_path='C:/Users/eberz/Documents/Projects for fun/Bonsai NN/',number_images=500):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)


In [8]:
search_and_download("bonsai tree", DRIVER_PATH)

Found: 100 search results. Extracting links from 0:100


KeyboardInterrupt: 

In [18]:
search_and_download("dwarf jade bonsai", DRIVER_PATH)

Found: 100 search results. Extracting links from 0:100
Found: 101 image links, done!
SUCCESS - saved https://media.istockphoto.com/photos/dwarf-jade-bonsai-tree-picture-id474448316 - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/dwarf_jade_bonsai\490eeb1529.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRkAa6DCyVpNt7ouT9KqH4Gox7GXqeAopcEfQ&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/dwarf_jade_bonsai\87fbe1202d.jpg
SUCCESS - saved https://cdn11.bigcommerce.com/s-6ys4nr/images/stencil/1280x1280/products/4355/20411/DIY-Dwarf-Jade_Premium__79812.1601486767.jpg?c=2 - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/dwarf_jade_bonsai\a5488c7695.jpg
SUCCESS - saved https://c8.alamy.com/comp/DFYG7Y/dwarf-jade-bonsai-tree-empress-garden-pune-maharashtra-india-DFYG7Y.jpg - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/dwarf_jade_bonsai\b0105b63bf.jpg
SUCCESS - saved https://i.ebayimg.com/images/g/hDUAAOSwnDZT9WU5/s-l640.jp

SUCCESS - saved https://cdn11.bigcommerce.com/s-6ys4nr/images/stencil/1280x1280/products/13657/45701/IMG_6972__68303.1601064412.jpg?c=2 - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/dwarf_jade_bonsai\314cb5fc14.jpg
SUCCESS - saved https://www.plantkart.com/wp-content/uploads/2017/04/IMG_1574-600x740.jpg - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/dwarf_jade_bonsai\44cf647837.jpg
SUCCESS - saved https://trustyflowers.com/store2/pub/media/catalog/product/cache/image/265x265/beff4985b56e3afdbeabfc89641a4582/d/w/dwarf-jade.jpg - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/dwarf_jade_bonsai\88760c11ac.jpg
SUCCESS - saved https://i.ytimg.com/vi/9rTZz2anKgY/hqdefault.jpg - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/dwarf_jade_bonsai\096841a22e.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTGTGZAhXEj6Dup82AOjeDzL5raFd3TRydaDg&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/dwarf_jade_bonsai\5e4aadb828.

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSZ5EeaTKj9pHPBEOTls-fgLKGzH4wkL2UhiQ&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/dwarf_jade_bonsai\f50f5eee63.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQbzUJiUECt6ERqdws3h2LfmilTWEvWOPE-Ag&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/dwarf_jade_bonsai\834fd8e1af.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTaPC0AAN7xdkDyRqHcPRcL_1nDudeiXyEz7g&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/dwarf_jade_bonsai\89d9c4ddeb.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRgR6Qk3ul3JhG6IzgM96Mq37KKbjFPoCemPA&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/dwarf_jade_bonsai\75e19490db.jpg
SUCCESS - saved https://cdn11.bigcommerce.com/s-bhqinueo9m/images/stencil/2048x2048/products/14779/8640/dwarf-jade-small-indoor-bonsai-dt9016dj-3__47254.1567627170.jpg?c=2 - as

In [9]:
words2search = [#'ficus bonsai', 'juniper bonsai', 'japanese maple bonsai', 'trident maple bonsai',
               #'pomegranate bonsai', 'chinese elm bonsai', 'zelkova bonsai', 'hornbeam bonsai',
               #'wisteria bonsai', 'magnolia bonsai', 'crabapple bonsai', 'oak bonsai',
               #'celtis bonsai', 'jacaranda bonsai', 'chinese pepper bonsai', 'ginko bonsai',
               #'adenium bonsai', 'japanese winterberry bonsai', 'japanese cherry bonsai',
               'weeping willow bonsai', 'flame tree bonsai', 'baobab bonsai', 'birch bonsai',
               'chinese quince bonsai', 'hawthorn bonsai', 'judas bonsai tree', 'privet bonsai',
               'boxwood bonsai', 'snow rose bonsai', 'olive bonsai', 'jade bonsai', 'fuchsia bonsai',
               'fig tree bonsai', 'fukien tea bonsai', 'azalea bonsai', 'bird plum bonsai',
               'bougainvillea bonsai', 'cotoneaster bonsai', 'money tree bonsai', 'gardenia bonsai',
               'crepe myrtle bonsai', 'brush cherry bonsai', 'hawaiian umrella bonsai', 'brazilian rain bonsai tree',
               'japanese holly bonsai', 'citrus bonsai', 'premna bonsai', 'myrtle bonsai', 'acacia bonsai',
               'firethorn bonsai', 'water jasmine bonsai', 'camellia bonsai', 'spruce bonsai', 
               'yew bonsai', 'buddhist pine bonsai', 'cedar bonsai', 'pine bonsai', 'larch bonsai',
               'redwood bonsai', 'bald cypress bonsai', 'hinoki cypress bonsai']

In [10]:
for i in words2search:
    print(i)

weeping willow bonsai
flame tree bonsai
baobab bonsai
birch bonsai
chinese quince bonsai
hawthorn bonsai
judas bonsai tree
privet bonsai
boxwood bonsai
snow rose bonsai
olive bonsai
jade bonsai
fuchsia bonsai
fig tree bonsai
fukien tea bonsai
azalea bonsai
bird plum bonsai
bougainvillea bonsai
cotoneaster bonsai
money tree bonsai
gardenia bonsai
crepe myrtle bonsai
brush cherry bonsai
hawaiian umrella bonsai
brazilian rain bonsai tree
japanese holly bonsai
citrus bonsai
premna bonsai
myrtle bonsai
acacia bonsai
firethorn bonsai
water jasmine bonsai
camellia bonsai
spruce bonsai
yew bonsai
buddhist pine bonsai
cedar bonsai
pine bonsai
larch bonsai
redwood bonsai
bald cypress bonsai
hinoki cypress bonsai


In [11]:
# Create loop for words2search

for i in words2search:
    wd = webdriver.Chrome(executable_path=DRIVER_PATH)
    wd.get('https://www.google.com/imghp?hl=EN')
    search_box = wd.find_element_by_css_selector('input.gLFyf')
    search_box.send_keys(i)
    search_box.send_keys(Keys.RETURN)
    new_url = str(wd.current_url)

    def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
        def scroll_to_end(wd):
            wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(sleep_between_interactions)    
    
        # build the google query
        search_url = new_url

        # load the page
        wd.get(search_url.format(q=query))

        image_urls = set()
        image_count = 0
        results_start = 0
        while image_count < max_links_to_fetch:
            scroll_to_end(wd)

            # get all image thumbnail results
            thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
            number_results = len(thumbnail_results)

            print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")

            for img in thumbnail_results[results_start:number_results]:
                # try to click every thumbnail such that we can get the real image behind it
                try:
                    img.click()
                    time.sleep(sleep_between_interactions)
                except Exception:
                    continue

                # extract image urls    
                actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
                for actual_image in actual_images:
                    if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                        image_urls.add(actual_image.get_attribute('src'))

                image_count = len(image_urls)

                if len(image_urls) >= max_links_to_fetch:
                    print(f"Found: {len(image_urls)} image links, done!")
                    break
            else:
                print("Found:", len(image_urls), "image links, looking for more ...")
                time.sleep(10)
                #return
                load_more_button = wd.find_element_by_css_selector(".mye4qd")
                if load_more_button:
                    wd.execute_script("document.querySelector('.mye4qd').click();")

            # move the result startpoint further down
            results_start = len(thumbnail_results)

        return image_urls

    def persist_image(folder_path:str,url:str):
        try:
            image_content = requests.get(url).content

        except Exception as e:
            print(f"ERROR - Could not download {url} - {e}")

        try:
            image_file = io.BytesIO(image_content)
            image = Image.open(image_file).convert('RGB')
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
            with open(file_path, 'wb') as f:
                image.save(f, "JPEG", quality=85)
            print(f"SUCCESS - saved {url} - as {file_path}")
        except Exception as e:
            print(f"ERROR - Could not save {url} - {e}")

            # Function that puts the above two all together
    def search_and_download(search_term:str,driver_path:str,target_path='C:/Users/eberz/Documents/Projects for fun/Bonsai NN/',number_images=300):
        target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

        if not os.path.exists(target_folder):
            os.makedirs(target_folder)

        with webdriver.Chrome(executable_path=driver_path) as wd:
            res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.1)

        for elem in res:
            persist_image(target_folder,elem)

    search_and_download(str(i), DRIVER_PATH)
    wd.quit()


Found: 100 search results. Extracting links from 0:100
Found: 81 image links, looking for more ...
Found: 212 search results. Extracting links from 100:212
Found: 194 image links, looking for more ...
Found: 312 search results. Extracting links from 212:312
Found: 301 image links, done!
SUCCESS - saved https://www.bonsaiboy.com/catalog/media/w033.jpg - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/weeping_willow_bonsai\ef77de25f7.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTEUIVMgV7ZdFW40tdU1PHqn5TTD4ecjZsVSQ&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/weeping_willow_bonsai\e6f4821d48.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR9K47XOGNEV68CX_PS0rzohHkvY92vZsifKg&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/weeping_willow_bonsai\b018adc6e5.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSReqi9TJcz5VjuGX1c4yaxrVPo4e47UaxgPQ&usqp=CAU - as C:/Users/ebe

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRMyM3RiK8AR--LJ8LMP1WY8KJTfK7u7EDbbg&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/weeping_willow_bonsai\3888ccb924.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRdzpiapyuNIVkxzn6c7peB_DT1fGx6illR5g&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/weeping_willow_bonsai\b34909802d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQtIUH6qVTnB-LsXUZ5biiQPV6pkcUNPoNY5A&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/weeping_willow_bonsai\716392fb60.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRGiF3-lbXGWpvXmtrrMWwoHt7r5Djy10eu-w&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/weeping_willow_bonsai\9cae8e2e19.jpg
SUCCESS - saved https://i.ytimg.com/vi/Jo1TwhEViYM/maxresdefault.jpg - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/weeping_willow_bonsai\95e872dd09.j

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRvo40-B_tUWTb3oM7525zNj7a7X1qdl8B7pw&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/weeping_willow_bonsai\acfc89bff0.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS3wLHD7s-Dpe2vIsY4OsoOVRVKZC-_ThIDNw&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/weeping_willow_bonsai\c69dcad158.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSqjoZbbwLqJ0FxbAkNmOU6iQQLP_YnSYSA0A&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/weeping_willow_bonsai\103d236759.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT4Vd8XB8rVjLrlLrBNsVIKjr1kSFI1LfzzSA&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/weeping_willow_bonsai\e46fd047d1.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ0djIXckm0CC1Jc5Wd1QL4h7tsy5PpzycOWA&usqp=CAU - as C:/Users/eberz/Documents/Projects for f

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRYeD9ZD0IqnnW9LhcGnhj406_UdvXqllpC0g&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/weeping_willow_bonsai\5d28ed148c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTvsTtOCQgD9GeBHnrPMrx_gQS5WYVJM9x9Ag&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/weeping_willow_bonsai\536a938159.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRu6_fcusmobqZFah5o5SO5y98fPy99KSMubA&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/weeping_willow_bonsai\32771fcfb9.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR3E3TA2O8Sstm9_ZlcLrT9a4i7VNoU3QLI1A&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/weeping_willow_bonsai\313a9e7410.jpg
SUCCESS - saved https://m.media-amazon.com/images/I/515U8ke7b9L._AC_SS450_.jpg - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/weeping_willow_bonsai\4c

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQqASidzt-2elc9aZzGgq6JXreOtlH-FqViGg&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/weeping_willow_bonsai\63004c503f.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ8Iox_AiGJ8PH8WgOSeByvlZeJdWksIU4tOA&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/weeping_willow_bonsai\a6f0bf15fc.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTcWRcuwGHKTWe3imFO5hCLTYOgLAQdpPTifg&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/weeping_willow_bonsai\011a6d46c7.jpg
SUCCESS - saved https://548038.smushcdn.com/934934/wp-content/uploads/2015/04/ginseng-ficus-bonsai-tree.jpg?lossy=1&strip=0&webp=1 - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/weeping_willow_bonsai\95dee2e7b2.jpg
SUCCESS - saved https://www.bonsaiboy.com/catalog/media/w022.jpg - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/weeping_willow_bonsai\9

...

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTEMNqwMn1UXNH0pamtCQpKwc9F72jfQARHqg&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/hinoki_cypress_bonsai\398e8c7858.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSWfc7uW5Pfw-7QXF8VX8MGmQeXxf_XZOPM_g&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/hinoki_cypress_bonsai\033c4a4273.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQwxRS_kfnVulM2lkbU0WAReUQYpYoqK6n3aA&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/hinoki_cypress_bonsai\04d0a2136b.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSIQdVtdmzymrQ0pSJcgD5-PqHSpXbcAlNkTw&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/hinoki_cypress_bonsai\2364159e9c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSCRMPoWMsaZCgewO2n80KP92HIsVLcGeFJQw&usqp=CAU - as C:/Users/eberz/Documents/Projects for f

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT8v4ihKb6sC78PsyLbrb6nIpFfo0ogBME4hQ&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/hinoki_cypress_bonsai\8edd6aed7c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRhx2sQl2lS7bxFnl2ff6so7WjU_POhb5HkuA&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/hinoki_cypress_bonsai\1ff84e4095.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRnJo416kajisEvfPTSX8jTaqRo1LBrJylAGQ&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/hinoki_cypress_bonsai\4b0b6d77e8.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQKZ5UYGqUY4g-KxF2YSZt6rG0vhLnnxRa98g&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/hinoki_cypress_bonsai\7542a998f7.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRgly0hy9luutK6phzpVptUD2lym0VHRBL2Wg&usqp=CAU - as C:/Users/eberz/Documents/Projects for f

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRPdczi30S2r1Nq9BK92LUiiyw9HMOz1T-cow&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/hinoki_cypress_bonsai\ac127f6fde.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQYPsI8v6uz0IgAl_6A2gZvqcIDf0wBEQ1hvg&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/hinoki_cypress_bonsai\7576e544e4.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTCgACNbpLXb7jTGpJCkMy1vNrsnhj5UgLSKg&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/hinoki_cypress_bonsai\260ac7068a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcREeEDJBmTaYzdCjJEPfA2wHlLqGhkCbAFNFQ&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/hinoki_cypress_bonsai\537ebd17bc.jpg
SUCCESS - saved https://t1.thpservices.com/previewimage/gallil/e4b521bb94a41962e649e99f257b9054/afl-qcza003824.jpg - as C:/Users/eberz/Documents/Projects for fu

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRjdBePeW7d9pZOF8ZewHixflVoHh63KZz0pQ&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/hinoki_cypress_bonsai\617fb26395.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS6TcgfBpdI9HhXGe81EVz9wU1dclh5EzJO1g&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/hinoki_cypress_bonsai\a47fbf2fa9.jpg
SUCCESS - saved https://valuabletoknow.com/wp-content/uploads/2018/10/bonsai-1129601_1920-min-232x300.jpg - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/hinoki_cypress_bonsai\9c489edbd7.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTxEYn6Ty3y5TIRTrErrMVu0VkI7WPWnUx5wg&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/hinoki_cypress_bonsai\363f88e737.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTiMegNPILDbIFaTNaI1fvoh0ZCbb6K1SK4WA&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai 

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSa6765TRI22x_MsIVRE2QUPY19UbRIZ5OBqQ&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/hinoki_cypress_bonsai\a88549a4c3.jpg
SUCCESS - saved https://i.pinimg.com/originals/de/7b/c6/de7bc6d80abfcad3366130b6373b5982.jpg - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/hinoki_cypress_bonsai\e0fcf3570f.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQxeIPp-InG0c2bF2b32LYyAjv3J6fulU8UKw&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/hinoki_cypress_bonsai\2f62155e21.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSYTS-P6U5K0iOgk_1sd0HQJ7OZhwBvPaSjpw&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/hinoki_cypress_bonsai\20dfb6fc30.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ0Ct0ShUt6OB09mcxCi_0ONTMT3HMMacaUtg&usqp=CAU - as C:/Users/eberz/Documents/Projects for fun/Bonsai NN/hinoki_cyp